In [100]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager
import concurrent.futures
import string
import os
import time
import pandas as pd
import json
import openpyxl
from tqdm import tqdm


# CONFIGURACION

In [2]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-blink-features=AUtomationControlled")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")

In [3]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options) # Esto lo defino dentro de las funciones para que se inicialice
driver.quit()

# URLs DE BINDER

In [4]:
# Incubadoras -Hecho
# Ultracongeladores -Hecho
# Camaras Secado - Hecho
# Camara de CLima - Hecho

incubadorasUrls = ["https://www.binder-world.com/int-es/productos/incubadoras/incubadoras-estandar/lista-productos-incubadoras-estandar",
"https://www.binder-world.com/int-es/productos/incubadoras/incubadoras-refrigeradas/lista-productos-incubadoras-refrigeradas",
"https://www.binder-world.com/int-es/productos/incubadoras/incubadoras-co2/lista-productos-incubadoras-co2"]

ultracongeladores = ["https://www.binder-world.com/int-es/productos/ultracongeladores/lista-productos-ultracongeladores"]

camarasSecado = ["https://www.binder-world.com/int-es/productos/camaras-de-secado/lista-productos-secado-temperacion","https://www.binder-world.com/int-es/productos/camaras-de-secado/lista-productos-secado-temperacion?ai%5Baction%5D=lists&ai%5Bcontroller%5D=Catalog&ai%5Bl_page%5D=2"]
camarasDeClima = ["https://www.binder-world.com/int-es/productos/camaras-climaticas/camaras-clima-constante/lista-productos-clima-constante","https://www.binder-world.com/int-es/productos/camaras-climaticas/camaras-clima-variable/lista-productos-clima-variable",
                  "https://www.binder-world.com/int-es/productos/camaras-climaticas/camaras-walk-in/lista-de-productos-de-camaras-walk-in"]




In [5]:
# Una sola url para hacer pruebas de todo el proceso
url = "https://www.binder-world.com/int-es/productos/incubadoras/incubadoras-estandar/lista-productos-incubadoras-estandar"
print(url.split("/")[-2])


# Urls de toda la pagina
listaUrls = [incubadorasUrls,ultracongeladores,camarasSecado, camarasDeClima]

incubadoras-estandar


# FUNCION PARA VISUALIZAR HTML EN PRIMERA INSTANCIA

In [6]:

# solo para ver la estructura
def scrape_products_html(url):
    driver = webdriver.Chrome(service=service, options=chrome_options) # Esto lo defino dentro de las funciones para que se inicialice

    driver.get(url)

    # Espera hasta que los elementos .product-item estén cargados
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product-item")))

    products = driver.find_elements(By.CSS_SELECTOR, ".product-item")
    
    prod_list = []
    for product in products:
        # Extraemos el HTML completo de cada producto para inspeccionarlo
        prod_html = product.get_attribute('outerHTML')
        prod_list.append(prod_html)
    
    driver.quit()
    return prod_list

# Uso
url = "https://www.binder-world.com/int-es/productos/incubadoras/incubadoras-estandar/lista-productos-incubadoras-estandar"
html_list = scrape_products_html(url)

# Guardamos el HTML para inspección
with open("productsPrueba.html", "w", encoding="utf-8") as f:
    f.write("\n".join(html_list))

print("Se ha guardado el HTML de los productos.")

Se ha guardado el HTML de los productos.


## CONSEGUIR DATOS GENERALES DE CADA PRODUCTO Y GUARDARLOS EN UN JSON:

Datos que se guardan en el json:
- Modelo
- Categoria
- Nombre Modelo
- Url imagen
- Source URL (La pagina en la que aparece)
- Product URL --> Pagina en la que aparecen mas datos sobre producto/familia


In [8]:
def scrape_and_save_incremental(urls, filename="products.json"):
    """
    Procesa cada URL y guarda los datos incrementalmente en un archivo JSON para evitar bloqueos.
    """
    try:
        # Cargar datos existentes, si el archivo ya existe
        try:
            with open(filename, "r", encoding="utf-8") as f:
                all_products = json.load(f)
        except FileNotFoundError:
            all_products = []  # Si no existe el archivo, inicializar lista vacía

        processed_urls = {item.get("source_url") for item in all_products}  # Seguimiento de URLs procesadas

        for url in urls:
            if url in processed_urls:
                print(f"Saltando URL ya procesada: {url}")
                continue

            print(f"Procesando URL: {url}")
            driver = webdriver.Chrome(service=service, options=chrome_options)
            time.sleep(3)
            driver.get(url)

            try:
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product-item")))
                products = driver.find_elements(By.CSS_SELECTOR, ".product-item")
                new_products = []

                for product in products:
                    print("Descansando 2 segundos...")
                    time.sleep(2)
                    try:
                        # Extraer datos básicos del producto
                        code = str(product.find_element(By.CSS_SELECTOR, ".product-item_code").text)
                        name = str(product.find_element(By.CSS_SELECTOR, ".name").text)
                        link = product.find_element(By.CSS_SELECTOR, "a.product-item_link").get_attribute("href")
                        categoria = str(url.split("/")[-2])
                        image_url = product.find_element(By.CSS_SELECTOR, ".product-item_img img").get_attribute("src")
                        code_for_url = code.replace(" ", "-")
                        product_page_url = "/".join(str(url).split("/")[:-1]) + "/producto/" + code_for_url.lower()

                        # Agregar producto a la lista temporal
                        new_products.append({
                            "modelo": code,
                            "categoria": categoria,
                            "nombre": name,
                            "image_url": image_url,
                            "product_page_url": product_page_url,
                            "source_url": url  # Registrar la URL de origen
                        })

                    except Exception as e:
                        print(f"Error al procesar producto: {e}")
                        continue

                # Extender la lista general y guardar en archivo
                all_products.extend(new_products)
                with open(filename, "w", encoding="utf-8") as f:
                    json.dump(all_products, f, ensure_ascii=False, indent=4)

            except Exception as e:
                print(f"Error al procesar URL {url}: {e}")
            finally:
                driver.quit()  # Asegurarse de cerrar el navegador

    except Exception as e:
        print(f"Error general: {e}")

In [19]:

# Incubadoras -Hecho
# Ultracongeladores -Hecho
# Camaras Secado - Hecho
# Camara de CLima - Hecho

incubadorasUrls = ["https://www.binder-world.com/int-es/productos/incubadoras/incubadoras-estandar/lista-productos-incubadoras-estandar",
"https://www.binder-world.com/int-es/productos/incubadoras/incubadoras-refrigeradas/lista-productos-incubadoras-refrigeradas",
"https://www.binder-world.com/int-es/productos/incubadoras/incubadoras-co2/lista-productos-incubadoras-co2"]

ultracongeladores = ["https://www.binder-world.com/int-es/productos/ultracongeladores/lista-productos-ultracongeladores"]

camarasSecado = ["https://www.binder-world.com/int-es/productos/camaras-de-secado/lista-productos-secado-temperacion","https://www.binder-world.com/int-es/productos/camaras-de-secado/lista-productos-secado-temperacion?ai%5Baction%5D=lists&ai%5Bcontroller%5D=Catalog&ai%5Bl_page%5D=2"]
camarasDeClima = ["https://www.binder-world.com/int-es/productos/camaras-climaticas/camaras-clima-constante/lista-productos-clima-constante","https://www.binder-world.com/int-es/productos/camaras-climaticas/camaras-clima-variable/lista-productos-clima-variable",
                  "https://www.binder-world.com/int-es/productos/camaras-climaticas/camaras-walk-in/lista-de-productos-de-camaras-walk-in"]


scrape_and_save_incremental(urls=camarasDeClima)


Saltando URL ya procesada: https://www.binder-world.com/int-es/productos/camaras-climaticas/camaras-clima-constante/lista-productos-clima-constante
Saltando URL ya procesada: https://www.binder-world.com/int-es/productos/camaras-climaticas/camaras-clima-variable/lista-productos-clima-variable
Saltando URL ya procesada: https://www.binder-world.com/int-es/productos/camaras-climaticas/camaras-walk-in/lista-de-productos-de-camaras-walk-in


Ahora tenemos todos los productos en general, con link a cada product_page_url desde donde voy a conseguir mas datos

## EXTRAER DATOS ESPECIFICO DE CADA PRODUCTO/FAMILIA

In [81]:


def procesar_productos_y_actualizar_json(data_file="products.json", output_file="productos_combinados.json"):
    """
    Procesa cada producto en products.json, obtiene datos adicionales de product_page_url
    y actualiza progresivamente un archivo JSON combinado.
    """
    # Cargar datos básicos del archivo products.json
    try:
        with open(data_file, "r", encoding="utf-8") as f:
            products = json.load(f)
    except FileNotFoundError:
        print(f"El archivo {data_file} no existe.")
        return

    # Cargar datos existentes del archivo combinado si ya existe
    if os.path.exists(output_file):
        with open(output_file, "r", encoding="utf-8") as f:
            combined_data = json.load(f)
    else:
        combined_data = []

    # Crear un índice de productos procesados previamente
    processed_product_ids = {
        f"{p['modelo']}": True
        for p in combined_data
    }

    # Usar tqdm para mostrar el progreso
    for product in tqdm(products, desc="Procesando productos", unit="producto"):
        try:
            modelo = product.get("modelo")
            product_url = product.get("product_page_url")
            categoria = product.get("categoria")

            if not product_url or not modelo:
                print(f"Producto inválido o sin URL: {product}")
                continue

            # Generar un identificador único para el producto
            product_id = f"{modelo}"

            # Saltar productos ya procesados
            if product_id in processed_product_ids:
                continue

            # Obtener datos adicionales del producto
            specific_data = scrapear_datos_especificos(product_url)

            # Combinar los datos básicos y específicos
            combined_product = {
                **product,
                **specific_data,
            }

            # Guardar al archivo combinado inmediatamente
            combined_data.append(combined_product)
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(combined_data, f, ensure_ascii=False, indent=4)

            # Marcar como procesado
            processed_product_ids[product_id] = True
        except Exception as e:
            print(f"Error al procesar el producto {product.get('modelo')}: {e}")

    print(f"Procesamiento completo. Datos combinados guardados en {output_file}.")



def scrapear_datos_especificos(urlProducto):
    """
    Extrae datos específicos de la página de un producto, asegurándose de expandir
    todas las secciones colapsadas antes de buscar datos en tablas.
    """
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(urlProducto)
    time.sleep(3)

    prod_data = {}

    # Extraer Descripción del producto
    try:
        descripcion = driver.find_element(By.XPATH, '//div[@class="col-12 col-xl-10 offset-xl-1 frame frame-space-after-medium frame-space-before-medium"]//p[@class="text-center lead"]').text
        prod_data['description'] = descripcion
    except Exception:
        prod_data['description'] = None

    # Extraer Rango de temperatura y otros detalles clave (product-key-fact)
    try:
        key_facts = driver.find_elements(By.XPATH, '//div[@class="product-key-fact"]//ul//li')
        key_facts_text = "|".join([fact.text.strip() for fact in key_facts if fact.text.strip()])
        prod_data['key_facts'] = key_facts_text if key_facts_text else None
    except Exception as e:
        print(f"Error al extraer los key facts: {e}")
        prod_data['key_facts'] = None
    
    # Conseguir catalogo, estudio de caso y manueales de instrucciones
    try:
        # Buscar enlaces que contienen "asset"
        href_elements = driver.find_elements(By.XPATH, '//a[contains(@href, "asset")]')
        hrefs = [element.get_attribute('href').strip() for element in href_elements]

        # Separar los enlaces según el orden esperado
        catalogo_link = hrefs[0] if len(hrefs) > 0 else None
        estudio_caso_link = hrefs[1] if len(hrefs) > 1 else None
        manual_instrucciones_link = hrefs[2] if len(hrefs) > 2 else None

        # Guardar en prod_data
        prod_data['catalogo'] = catalogo_link
        prod_data['estudio_de_caso'] = estudio_caso_link
        prod_data['manual_de_instrucciones'] = manual_instrucciones_link
    except Exception as e:
        print(f"Error al extraer y organizar los enlaces href: {e}")
        prod_data['catalogo'] = None
        prod_data['estudio_de_caso'] = None
        prod_data['manual_de_instrucciones'] = None
    # Extraer Tabla "Número de Artículo" y "Modelo"
    try:
        modelo_numeroDeArticulo = []
        table_element = driver.find_element(By.XPATH, '//div[@class="product-tech-detail"]//table[@class="table"]')
        rows = table_element.find_elements(By.TAG_NAME, 'tr')

        if rows and len(rows) > 1:
            header_cols = rows[0].find_elements(By.TAG_NAME, 'td')
            modelos = [col.text.strip() for col in header_cols[1:] if col.text.strip()]

            for row in rows[1:]:
                cols = row.find_elements(By.TAG_NAME, 'td')
                if len(cols) > 1:
                    descripcion = cols[0].text.strip()
                    for i, modelo in enumerate(modelos):
                        if i + 1 < len(cols):
                            valor = cols[i + 1].text.strip()
                            if descripcion.lower() == "número de artículo" and valor:
                                modelo_numeroDeArticulo.append({"modelo": modelo, "numero_articulo": valor})

        prod_data['modelo_numeroDeArticulo'] = modelo_numeroDeArticulo if modelo_numeroDeArticulo else None
    except Exception:
        prod_data['modelo_numeroDeArticulo'] = None


    # Expandir todas las secciones colapsadas (accordion-icon)
    try:
        accordion_icons = driver.find_elements(By.CSS_SELECTOR, ".accordion-icon")
        #print(f"Numero de accordion_icons: {len(accordion_icons)}")
        for icon in accordion_icons:
            driver.execute_script("arguments[0].click();", icon)
            time.sleep(1)  # Pequeño retraso para permitir que la sección se expanda
    except Exception as e:
        print(f"Error al intentar expandir secciones: {e}")

    # Extraer Tabla de Datos Generales
    try:
        models_data = {}

        # Esperar hasta que las tablas estén disponibles
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="product-tech-detail"]//table[@class="table"]')))

        # Encuentra todas las tablas
        table_elements = driver.find_elements(By.XPATH, '//div[@class="product-tech-detail"]//div[@class="accordion-item"]//div[@class="accordion-collapse collapse show"]//div[@class="accordion-body"]//table[@class="table"]')

        for table in table_elements:
            rows = table.find_elements(By.TAG_NAME, 'tr')
            if not rows:
                continue

            header_cols = rows[0].find_elements(By.TAG_NAME, 'td')
            modelos = [col.text.strip() for col in header_cols[1:] if col.text.strip()]

            for row in rows[1:]:
                cols = row.find_elements(By.TAG_NAME, 'td')
                if len(cols) < 2:
                    continue

                descripcion = cols[0].text.strip()

                for i, modelo in enumerate(modelos):
                    if i + 1 < len(cols):
                        valor = cols[i + 1].text.strip()

                        if valor:  # Verifica si hay valor en la celda
                            if modelo not in models_data:
                                models_data[modelo] = []
                            models_data[modelo].append({"descripcion": descripcion, "valor": valor})

        prod_data['models_data'] = models_data if models_data else None

    except Exception as e:
        print(f"Error al procesar la tabla: {e}")
        prod_data['models_data'] = None

    driver.quit()
    return prod_data


In [82]:
procesar_productos_y_actualizar_json()

Procesando productos:  97%|█████████▋| 104/107 [40:13<01:10, 23.56s/producto] 

Error al procesar la tabla: Message: 



Procesando productos:  98%|█████████▊| 105/107 [40:31<00:43, 21.88s/producto]

Error al procesar la tabla: Message: 



Procesando productos:  99%|█████████▉| 106/107 [40:49<00:20, 20.89s/producto]

Error al procesar la tabla: Message: 



Procesando productos: 100%|██████████| 107/107 [41:07<00:00, 23.06s/producto]

Procesamiento completo. Datos combinados guardados en productos_combinados.json.


## CREAR UN CSV CON LOS DATOS DE CADA PRODUCTO

Para cada producto, tengo que cojer todos los valores hasta llegar a modelo_numeroDeArticulo


In [96]:
import json
from pprint import pprint

try:
    with open("productos_combinados.json", "r", encoding="utf-8") as f:
        products = json.load(f)
        # Usar pprint para mostrar la estructura del JSON
        pprint(products)
except FileNotFoundError:
    print(f"El archivo {'productos_combinados.json'} no existe.")
except json.JSONDecodeError:
    print("El archivo no contiene un JSON válido.")

data = []

for p in products:
    # Datos comunes
    modelo = p.get("modelo")
    categoria = p.get("categoria")
    nombre = p.get("nombre")
    image_url = p.get("image_url")
    product_page_url = p.get("product_page_url")
    source_url = p.get("source_url")
    description = p.get("description")
    key_facts = p.get("key_facts")
    catalogo = p.get("catalogo")
    estudio_de_caso = p.get("estudio_de_caso")
    manual_de_instrucciones = p.get("manual_de_instrucciones")

    mapa_modelo_numeroArticulo = p.get("modelo_numeroDeArticulo", [])
    # Verificar si mapa_modelo_numeroArticulo es una lista
    if isinstance(mapa_modelo_numeroArticulo, list):
        for mapa in mapa_modelo_numeroArticulo:
            modelo_especifico = mapa.get("modelo")
            numero_de_articulo = mapa.get("numero_articulo")

            # Ahora conseguir los datos para cada modelo especifico
            models_data = p.get("models_data", [])

            for m in models_data:
                # Verificar si m es un diccionario y contiene modelo_especifico
                if isinstance(m, dict) and modelo_especifico in m:
                    detalles = m[modelo_especifico]
                    
                    # Crear el diccionario base
                    producto_data = {
                        "modelo": modelo,
                        "categoria": categoria,
                        "nombre": nombre,
                        "image_url": image_url,
                        "product_page_url": product_page_url,
                        "source_url": source_url,
                        "description": description,
                        "key_facts": key_facts,
                        "catalogo": catalogo,
                        "estudio_de_caso": estudio_de_caso,
                        "manual_de_instrucciones": manual_de_instrucciones,
                        "modelo_especifico": modelo_especifico,
                        "numero_de_articulo": numero_de_articulo
                    }
                    
                    # Agregar detalles dinámicos
                    if isinstance(detalles, list):
                        for detail in detalles:
                            nombre_detail = detail.get("descripcion")
                            contenido = detail.get("valor")
                            if nombre_detail:
                                producto_data[nombre_detail] = contenido
                    
                    # Agregar el producto procesado a la lista
                    data.append(producto_data)
    else:
        print(f"Estructura inesperada en modelo_numeroDeArticulo: {mapa_modelo_numeroArticulo}")

# Convertir la lista de datos a un DataFrame
import pandas as pd

df = pd.DataFrame(data)

# Llenar valores faltantes con NaN
df = df.fillna(value=pd.NA)

# Guardar en un archivo CSV
df.to_csv("productos_procesados.csv", index=False, encoding="utf-8")

print("Datos procesados y guardados en productos_procesados.csv")


[{'catalogo': 'https://www.binder-world.com/int-es/productos/incubadoras/incubadoras-estandar/producto/asset/16522',
  'categoria': 'incubadoras-estandar',
  'description': 'Con las incubadoras de la serie B dispondrá de un equipo '
                 'fiable y eficaz con un sólido equipamiento a un precio '
                 'atractivo.',
  'estudio_de_caso': 'https://www.binder-world.com/int-es/productos/incubadoras/incubadoras-estandar/producto/asset/16529',
  'image_url': 'https://www.binder-world.com/contentserv/assets/es/1_main_image_GF/B_028_GF_Web_670x515.jpg?v=20241220023016',
  'key_facts': 'Rango de temperatura:\n'
               'de +30 °C a +70 °C|Termostato hidráulico-mecánico|Limitador de '
               'temperatura de clase 1|Puerta interior de vidrio de seguridad '
               '(ESG)|2 rejillas cromadas',
  'manual_de_instrucciones': 'https://www.binder-world.com/int-es/productos/incubadoras/incubadoras-estandar/producto/asset/16536',
  'modelo': 'B 28',
  'modelo_nu

In [ ]:

import json
import pandas as pd

# Cargar datos desde el archivo JSON
try:
    with open("productos_combinados.json", "r", encoding="utf-8") as f:
        products = json.load(f)
except FileNotFoundError:
    print("El archivo 'productos_combinados.json' no existe.")
    exit()
except json.JSONDecodeError:
    print("El archivo no contiene un JSON válido.")
    exit()

data = []

for p in products:
    # Datos comunes
    modelo = p.get("modelo")
    categoria = p.get("categoria")
    nombre = p.get("nombre")
    image_url = p.get("image_url")
    product_page_url = p.get("product_page_url")
    source_url = p.get("source_url")
    description = p.get("description")
    key_facts = p.get("key_facts")
    catalogo = p.get("catalogo")
    estudio_de_caso = p.get("estudio_de_caso")
    manual_de_instrucciones = p.get("manual_de_instrucciones")

    mapa_modelo_numeroArticulo = p.get("modelo_numeroDeArticulo")
    if not mapa_modelo_numeroArticulo:  # Validar si existe
        print(f"Producto omitido: {p.get('nombre', 'Sin nombre')} - modelo_numeroDeArticulo es None")
        continue

    models_data = p.get("models_data", {})
    for mapa in mapa_modelo_numeroArticulo:
        modelo_especifico = mapa.get("modelo")
        numero_de_articulo = mapa.get("numero_articulo")

        if modelo_especifico in models_data:
            detalles = models_data[modelo_especifico]
        else:
            print(f"Modelo específico '{modelo_especifico}' no encontrado en 'models_data'")
            detalles = []

        # Crear el diccionario base con los datos comunes
        producto_data = {
            "modelo": modelo,
            "categoria": categoria,
            "nombre": nombre,
            "image_url": image_url,
            "product_page_url": product_page_url,
            "source_url": source_url,
            "description": description,
            "key_facts": key_facts,
            "catalogo": catalogo,
            "estudio_de_caso": estudio_de_caso,
            "manual_de_instrucciones": manual_de_instrucciones,
            "modelo_especifico": modelo_especifico,
            "numero_de_articulo": numero_de_articulo,
        }

        # Agregar los detalles dinámicos
        for detail in detalles:
            nombre_detail = detail.get("descripcion")
            contenido = detail.get("valor")
            if nombre_detail:  # Solo agregar si el nombre es válido
                producto_data[nombre_detail] = contenido

        data.append(producto_data)

# Convertir la lista de datos a un DataFrame
df = pd.DataFrame(data)

# Llenar valores faltantes con NaN
df = df.fillna(value=pd.NA)

# Guardar en un archivo CSV
output_file = "productos_procesados.csv"
df.to_csv(output_file, index=False, encoding="utf-8")

df.to_excel("binder_scrapeo_guria.xlsx")
print(f"Datos procesados y guardados en {output_file}")


Producto omitido: Modelo CB 56 | Incubadoras de CO2 con esterilización por aire caliente y sensor de CO₂ con esterilización térmica - modelo_numeroDeArticulo es None
Producto omitido: Model WIC 1 | Walk-in-chambers para ensayos de estabilidad según la directiva ICH (Q1A)  - modelo_numeroDeArticulo es None
Producto omitido: Model WIC 2 | Walk-in-chambers para ensayos de estabilidad según la directiva ICH (Q1A)  - modelo_numeroDeArticulo es None
Producto omitido: Model WIC 3 | Walk-in-chambers para ensayos de estabilidad según la directiva ICH (Q1A)  - modelo_numeroDeArticulo es None
Datos procesados y guardados en productos_procesados.csv
